# Transaction Data Analysis

This notebook analyzes transaction patterns and creates a graph structure for money laundering detection.

In [42]:
!pip install polars
!pip install networkx
!pip install matplotlib
!pip install igraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 655.2 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [igraph]


## Setup

Install required library.

In [2]:
import polars as pl
import networkx as nx
import matplotlib.pyplot as plt

# df = pl.read_csv('data/HI-Small_Trans.csv')
# Try lazy frame
df = pl.read_csv('data/HI-Small_Trans.csv')

# Sample only 10% of full data for memory management, commented out if want full data
df = df.sample(fraction=0.10, with_replacement=False, seed=42).lazy()

## Load Data

Read transaction data from CSV file.

In [3]:
df.collect()

Timestamp,From Bank,Account,To Bank,Account_duplicated_0,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
str,i64,str,i64,str,f64,str,f64,str,str,i64
"""2022/09/08 19:55""",7,"""807688770""",220504,"""807E0A9A0""",504.6,"""UK Pound""",504.6,"""UK Pound""","""Credit Card""",0
"""2022/09/02 15:48""",10099,"""803F400C0""",24,"""803FBE550""",20835.05,"""Yen""",20835.05,"""Yen""","""Cash""",0
"""2022/09/10 13:41""",1547,"""80D13CC40""",2467,"""80D4D5A50""",587.72,"""Euro""",587.72,"""Euro""","""ACH""",0
"""2022/09/07 15:19""",15747,"""8118ABE40""",20226,"""8130A3320""",20085.98,"""US Dollar""",20085.98,"""US Dollar""","""Cheque""",0
"""2022/09/08 14:02""",126,"""8136E45C1""",126,"""813F46D51""",0.00859,"""Bitcoin""",0.00859,"""Bitcoin""","""Bitcoin""",0
…,…,…,…,…,…,…,…,…,…,…
"""2022/09/08 09:11""",1024,"""801881BF0""",410,"""8028D2D60""",125.93,"""Euro""",125.93,"""Euro""","""ACH""",0
"""2022/09/02 17:05""",2454,"""8022D39C0""",1412,"""802EE4B70""",3780.75,"""US Dollar""",3780.75,"""US Dollar""","""ACH""",0
"""2022/09/09 12:01""",3,"""800045510""",1267,"""8007C73D0""",14299.75,"""Yuan""",14299.75,"""Yuan""","""Credit Card""",0


In [4]:
df = df.with_columns(
    pl.col('Timestamp').str.strptime(pl.Datetime, format='%Y/%m/%d %H:%M')
)

## Data Preparation

Convert timestamp column to datetime format.

In [5]:
df.collect()

Timestamp,From Bank,Account,To Bank,Account_duplicated_0,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
datetime[μs],i64,str,i64,str,f64,str,f64,str,str,i64
2022-09-08 19:55:00,7,"""807688770""",220504,"""807E0A9A0""",504.6,"""UK Pound""",504.6,"""UK Pound""","""Credit Card""",0
2022-09-02 15:48:00,10099,"""803F400C0""",24,"""803FBE550""",20835.05,"""Yen""",20835.05,"""Yen""","""Cash""",0
2022-09-10 13:41:00,1547,"""80D13CC40""",2467,"""80D4D5A50""",587.72,"""Euro""",587.72,"""Euro""","""ACH""",0
2022-09-07 15:19:00,15747,"""8118ABE40""",20226,"""8130A3320""",20085.98,"""US Dollar""",20085.98,"""US Dollar""","""Cheque""",0
2022-09-08 14:02:00,126,"""8136E45C1""",126,"""813F46D51""",0.00859,"""Bitcoin""",0.00859,"""Bitcoin""","""Bitcoin""",0
…,…,…,…,…,…,…,…,…,…,…
2022-09-08 09:11:00,1024,"""801881BF0""",410,"""8028D2D60""",125.93,"""Euro""",125.93,"""Euro""","""ACH""",0
2022-09-02 17:05:00,2454,"""8022D39C0""",1412,"""802EE4B70""",3780.75,"""US Dollar""",3780.75,"""US Dollar""","""ACH""",0
2022-09-09 12:01:00,3,"""800045510""",1267,"""8007C73D0""",14299.75,"""Yuan""",14299.75,"""Yuan""","""Credit Card""",0


Disregard transaction with Payment Format "Reinvesment"

In [6]:
df = df.filter(pl.col("Payment Format") != "Reinvestment")

## Create Nodes

Build graph nodes from transactions with ID, sender, receiver, time, amount, and label.

In [8]:
nodes = df.with_row_index("node_id").select([
    pl.col("node_id"),
    pl.col("Account").alias("f_i"),                # From
    pl.col("Account_duplicated_0").alias("b_i"),   # Beneficiary
    pl.col("Timestamp").alias("t_i"),              # Time
    pl.col("Amount Received").alias("a_i"),        # Amount
    pl.col("Is Laundering")                        # Ground truth
])

In [9]:
nodes.collect()

node_id,f_i,b_i,t_i,a_i,Is Laundering
u32,str,str,datetime[μs],f64,i64
0,"""807688770""","""807E0A9A0""",2022-09-08 19:55:00,504.6,0
1,"""803F400C0""","""803FBE550""",2022-09-02 15:48:00,20835.05,0
2,"""80D13CC40""","""80D4D5A50""",2022-09-10 13:41:00,587.72,0
3,"""8118ABE40""","""8130A3320""",2022-09-07 15:19:00,20085.98,0
4,"""8136E45C1""","""813F46D51""",2022-09-08 14:02:00,0.00859,0
…,…,…,…,…,…
459546,"""801881BF0""","""8028D2D60""",2022-09-08 09:11:00,125.93,0
459547,"""8022D39C0""","""802EE4B70""",2022-09-02 17:05:00,3780.75,0
459548,"""800045510""","""8007C73D0""",2022-09-09 12:01:00,14299.75,0


## Create Edges

Connect transactions where one receiver becomes the sender in another transaction.

In [10]:
edges = nodes.join(
    nodes,
    left_on="b_i", 
    right_on="f_i",
    suffix="_d"
).rename({"node_id": "v_s", "node_id_d": "v_d"})

In [12]:
edges.collect()

v_s,f_i,b_i,t_i,a_i,Is Laundering,v_d,b_i_d,t_i_d,a_i_d,Is Laundering_d
u32,str,str,datetime[μs],f64,i64,u32,str,datetime[μs],f64,i64
109860,"""8000955D0""","""807688770""",2022-09-08 17:26:00,22.09,0,0,"""807E0A9A0""",2022-09-08 19:55:00,504.6,0
86445,"""803A621E0""","""803F400C0""",2022-09-06 07:10:00,290618.32,0,1,"""803FBE550""",2022-09-02 15:48:00,20835.05,0
67016,"""80D074610""","""80D13CC40""",2022-09-05 04:13:00,1.53,0,2,"""80D4D5A50""",2022-09-10 13:41:00,587.72,0
352111,"""8043486F0""","""80D13CC40""",2022-09-08 07:21:00,904.52,0,2,"""80D4D5A50""",2022-09-10 13:41:00,587.72,0
361152,"""8043486F0""","""80D13CC40""",2022-09-05 05:25:00,904.52,0,2,"""80D4D5A50""",2022-09-10 13:41:00,587.72,0
…,…,…,…,…,…,…,…,…,…,…
446930,"""812123D20""","""1004286A8""",2022-09-02 23:21:00,1081.63,0,459549,"""80831D6F0""",2022-09-02 11:15:00,198.5,0
454187,"""804206B40""","""1004286A8""",2022-09-02 08:08:00,86.62,0,459549,"""80831D6F0""",2022-09-02 11:15:00,198.5,0
458099,"""80AC1D6F0""","""1004286A8""",2022-09-09 00:59:00,45.42,0,459549,"""80831D6F0""",2022-09-02 11:15:00,198.5,0


## Time Delta Window 

In [13]:
timedelta = pl.duration(hours=24)

## Filter Edges

Keep only edges where the second transaction occurs within 24 hours after the first.

In [14]:
edges = edges.filter(
    (pl.col("t_i_d") > pl.col("t_i")) & 
    (pl.col("t_i_d") < pl.col("t_i") + timedelta)
)

## Temporal View Results

Display final nodes and edges.

In [15]:
print(nodes.collect())

shape: (459_551, 6)
┌─────────┬───────────┬───────────┬─────────────────────┬───────────┬───────────────┐
│ node_id ┆ f_i       ┆ b_i       ┆ t_i                 ┆ a_i       ┆ Is Laundering │
│ ---     ┆ ---       ┆ ---       ┆ ---                 ┆ ---       ┆ ---           │
│ u32     ┆ str       ┆ str       ┆ datetime[μs]        ┆ f64       ┆ i64           │
╞═════════╪═══════════╪═══════════╪═════════════════════╪═══════════╪═══════════════╡
│ 0       ┆ 807688770 ┆ 807E0A9A0 ┆ 2022-09-08 19:55:00 ┆ 504.6     ┆ 0             │
│ 1       ┆ 803F400C0 ┆ 803FBE550 ┆ 2022-09-02 15:48:00 ┆ 20835.05  ┆ 0             │
│ 2       ┆ 80D13CC40 ┆ 80D4D5A50 ┆ 2022-09-10 13:41:00 ┆ 587.72    ┆ 0             │
│ 3       ┆ 8118ABE40 ┆ 8130A3320 ┆ 2022-09-07 15:19:00 ┆ 20085.98  ┆ 0             │
│ 4       ┆ 8136E45C1 ┆ 813F46D51 ┆ 2022-09-08 14:02:00 ┆ 0.00859   ┆ 0             │
│ …       ┆ …         ┆ …         ┆ …                   ┆ …         ┆ …             │
│ 459546  ┆ 801881BF0 ┆ 8028D2D60 

## Second Order Graph Creation

### Edge creation

In [17]:
s_edges = edges.with_columns([
    pl.concat_str([pl.col("f_i"), pl.col("b_i")], separator="->").alias("v_s"),
    pl.concat_str([pl.col("b_i"), pl.col("b_i_d")], separator="->").alias("v_d")
]).select(["v_s", "v_d"])

In [18]:
s_edges.collect().head()

v_s,v_d
str,str
"""8000955D0->807688770""","""807688770->807E0A9A0"""
"""807AD7960->807AD6AA0""","""807AD6AA0->8086A3C50"""
"""8076DEBB0->807C29950""","""807C29950->807D5A090"""
"""810592760->810A3B4A0""","""810A3B4A0->811357C40"""
"""800089CB0->80609DC20""","""80609DC20->8060DB870"""


### Weight Calculation

In [19]:
# Count times where one node goes to another
s_edges_with_count = s_edges.group_by(["v_s", "v_d"]).agg([
    pl.len().alias("spec_count")
])

denom_P = s_edges.group_by("v_s").agg([
    pl.len().alias("s_to_any")
])

denom_P_prime = s_edges.group_by("v_d").agg([
    pl.len().alias("any_to_d")
])

In [22]:
print(s_edges_with_count.collect().head())

print(denom_P.collect().head())

print(denom_P_prime.collect().head())

shape: (5, 3)
┌──────────────────────┬──────────────────────┬────────────┐
│ v_s                  ┆ v_d                  ┆ spec_count │
│ ---                  ┆ ---                  ┆ ---        │
│ str                  ┆ str                  ┆ u32        │
╞══════════════════════╪══════════════════════╪════════════╡
│ 80C17A100->80C5CEF80 ┆ 80C5CEF80->80CA65B80 ┆ 1          │
│ 802949240->8027987E0 ┆ 8027987E0->80BAFB1F0 ┆ 1          │
│ 80061AF50->80061AF50 ┆ 80061AF50->80BE56630 ┆ 1          │
│ 801465E70->100428660 ┆ 100428660->80E6B56F0 ┆ 1          │
│ 80D229080->100428660 ┆ 100428660->810E7D6A0 ┆ 1          │
└──────────────────────┴──────────────────────┴────────────┘
shape: (5, 2)
┌──────────────────────┬──────────┐
│ v_s                  ┆ s_to_any │
│ ---                  ┆ ---      │
│ str                  ┆ u32      │
╞══════════════════════╪══════════╡
│ 801E016D0->801E18890 ┆ 1        │
│ 80EDE2570->80F437C50 ┆ 2        │
│ 80266DB40->80326CDA0 ┆ 1        │
│ 80FE1DB40->

In [23]:
s_edges_with_weight = s_edges_with_count.join(
    denom_P,
    on="v_s",
    how="left"
).join(
    denom_P_prime,
    on="v_d",
    how="left"
).with_columns([
    (pl.col("spec_count") / pl.col("s_to_any")).alias("P"),
    (pl.col("spec_count") / pl.col("any_to_d")).alias("P_prime")
]).with_columns([
    (pl.max_horizontal([pl.col("P"), pl.col("P_prime")])).alias("weight")
])

In [24]:
print(s_edges_with_weight.collect().head())

shape: (5, 8)
┌───────────────┬──────────────┬────────────┬──────────┬──────────┬──────────┬──────────┬──────────┐
│ v_s           ┆ v_d          ┆ spec_count ┆ s_to_any ┆ any_to_d ┆ P        ┆ P_prime  ┆ weight   │
│ ---           ┆ ---          ┆ ---        ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
│ str           ┆ str          ┆ u32        ┆ u32      ┆ u32      ┆ f64      ┆ f64      ┆ f64      │
╞═══════════════╪══════════════╪════════════╪══════════╪══════════╪══════════╪══════════╪══════════╡
│ 80BE26900->10 ┆ 100428660->8 ┆ 1          ┆ 748      ┆ 38       ┆ 0.001337 ┆ 0.026316 ┆ 0.026316 │
│ 0428660       ┆ 0A71A250     ┆            ┆          ┆          ┆          ┆          ┆          │
│ 8082338C0->10 ┆ 100428660->8 ┆ 1          ┆ 974      ┆ 53       ┆ 0.001027 ┆ 0.018868 ┆ 0.018868 │
│ 0428660       ┆ 0161A8D0     ┆            ┆          ┆          ┆          ┆          ┆          │
│ 80352F5A0->10 ┆ 100428660->8 ┆ 1          ┆ 2145     ┆ 84       ┆ 0.000466 

### Apply second order graph's weight to Temporal graph's edges

In [25]:
edges_with_weight = edges.with_columns([
    pl.concat_str([pl.col("f_i"), pl.col("b_i")], separator="->").alias("tx_s"),
    pl.concat_str([pl.col("b_i"), pl.col("b_i_d")], separator="->").alias("tx_d")
]).join(
    s_edges_with_weight.select([
        "v_s", "v_d", "weight"
    ]),
    left_on=["tx_s", "tx_d"],
    right_on=["v_s", "v_d"],
    how="left"
).drop([
    "tx_s", "tx_d", "v_s", "v_d"
])

In [28]:
edges_with_weight.collect().filter(pl.col("Is Laundering") == 1)

f_i,b_i,t_i,a_i,Is Laundering,b_i_d,t_i_d,a_i_d,Is Laundering_d,weight
str,str,datetime[μs],f64,i64,str,datetime[μs],f64,i64,f64
"""8119F8CC0""","""80B0C3F40""",2022-09-11 14:02:00,8019.98,1,"""80B0C3F40""",2022-09-12 10:34:00,20956.02,0,1.0
"""8014E85C0""","""8007C82A0""",2022-09-03 19:10:00,3169.29,1,"""80309F260""",2022-09-04 11:13:00,777331.33,0,1.0
"""80BE87020""","""8013F0350""",2022-09-07 22:10:00,5472.94,1,"""8081AC860""",2022-09-08 12:40:00,147.31,0,1.0
"""804093380""","""811B90FA0""",2022-09-12 18:10:00,14732.08,1,"""811B90FA0""",2022-09-13 12:05:00,13835.02,0,1.0
"""806E1B6F0""","""8013F0350""",2022-09-07 11:22:00,1456.96,1,"""8081AC860""",2022-09-07 14:54:00,147.31,0,1.0
…,…,…,…,…,…,…,…,…,…
"""811D80C30""","""811A65E30""",2022-09-07 05:00:00,26259.53,1,"""811B5FB20""",2022-09-07 15:14:00,1180.16,0,1.0
"""8001B2240""","""80017C540""",2022-09-07 18:18:00,12959.17,1,"""8009ABB60""",2022-09-08 02:05:00,202.96,0,0.5
"""80693AB90""","""800049680""",2022-09-05 07:38:00,11625.36,1,"""808E40150""",2022-09-05 10:06:00,414.02,0,1.0


In [34]:
edges_with_weight.collect().filter(pl.col("Is Laundering") == 1)

f_i,b_i,t_i,a_i,Is Laundering,b_i_d,t_i_d,a_i_d,Is Laundering_d,weight
str,str,datetime[μs],f64,i64,str,datetime[μs],f64,i64,f64
"""8119F8CC0""","""80B0C3F40""",2022-09-11 14:02:00,8019.98,1,"""80B0C3F40""",2022-09-12 10:34:00,20956.02,0,1.0
"""8014E85C0""","""8007C82A0""",2022-09-03 19:10:00,3169.29,1,"""80309F260""",2022-09-04 11:13:00,777331.33,0,1.0
"""80BE87020""","""8013F0350""",2022-09-07 22:10:00,5472.94,1,"""8081AC860""",2022-09-08 12:40:00,147.31,0,1.0
"""804093380""","""811B90FA0""",2022-09-12 18:10:00,14732.08,1,"""811B90FA0""",2022-09-13 12:05:00,13835.02,0,1.0
"""806E1B6F0""","""8013F0350""",2022-09-07 11:22:00,1456.96,1,"""8081AC860""",2022-09-07 14:54:00,147.31,0,1.0
…,…,…,…,…,…,…,…,…,…
"""811D80C30""","""811A65E30""",2022-09-07 05:00:00,26259.53,1,"""811B5FB20""",2022-09-07 15:14:00,1180.16,0,1.0
"""8001B2240""","""80017C540""",2022-09-07 18:18:00,12959.17,1,"""8009ABB60""",2022-09-08 02:05:00,202.96,0,0.5
"""80693AB90""","""800049680""",2022-09-05 07:38:00,11625.36,1,"""808E40150""",2022-09-05 10:06:00,414.02,0,1.0


In [36]:
edges_with_weight.collect().filter(pl.col("Is Laundering") == 1)["weight"].value_counts().sort("weight")

weight,count
f64,u32
0.25,1
0.333333,1
0.5,6
0.666667,2
1.0,91


In [37]:
edges_with_weight.collect().filter(pl.col("Is Laundering") == 0)["weight"].value_counts().sort("weight")

weight,count
f64,u32
0.004878,32
0.00495,4
0.005236,1
0.005263,16
0.00578,48
…,…
0.846154,11
0.857143,72
0.875,21


In [ ]:
df_for_leiden = edges_with_weight.select([
    pl.concat_str(["f_i", "b_i", "t_i"]).alias("source_node"),
    pl.concat_str(["b_i", "b_i_d", "t_i_d"]).alias("target_node"),
    pl.col("weight")
])

In [33]:
df_for_leiden.collect()

source_node,target_node,weight
str,str,f64
"""8000955D08076887702022-09-08 1…","""807688770807E0A9A02022-09-08 1…",1.0
"""807AD7960807AD6AA02022-09-09 1…","""807AD6AA08086A3C502022-09-09 1…",1.0
"""8076DEBB0807C299502022-09-03 1…","""807C29950807D5A0902022-09-03 2…",0.5
"""810592760810A3B4A02022-09-05 1…","""810A3B4A0811357C402022-09-06 0…",1.0
"""800089CB080609DC202022-09-01 0…","""80609DC208060DB8702022-09-01 1…",1.0
…,…,…
"""802849B901004286A82022-09-02 0…","""1004286A880831D6F02022-09-02 1…",0.027778
"""8134D5C801004286A82022-09-02 0…","""1004286A880831D6F02022-09-02 1…",0.027778
"""80C523AF01004286A82022-09-02 0…","""1004286A880831D6F02022-09-02 1…",0.027778


In [38]:
df_for_leiden_with_weight_filtered = df_for_leiden.filter(pl.col("weight") >= 0.5)

In [41]:
df_for_leiden_with_weight_filtered.collect()

source_node,target_node,weight
str,str,f64
"""8000955D08076887702022-09-08 1…","""807688770807E0A9A02022-09-08 1…",1.0
"""807AD7960807AD6AA02022-09-09 1…","""807AD6AA08086A3C502022-09-09 1…",1.0
"""8076DEBB0807C299502022-09-03 1…","""807C29950807D5A0902022-09-03 2…",0.5
"""810592760810A3B4A02022-09-05 1…","""810A3B4A0811357C402022-09-06 0…",1.0
"""800089CB080609DC202022-09-01 0…","""80609DC208060DB8702022-09-01 1…",1.0
…,…,…
"""8077CA2A08063DECB02022-09-05 2…","""8063DECB0809CDBAC02022-09-06 1…",1.0
"""801EC5060801EE4A002022-09-09 0…","""801EE4A00810A99EB02022-09-09 1…",1.0
"""805B92690806D0D5802022-09-02 0…","""806D0D58080FDEB8002022-09-02 1…",1.0


In [49]:
import igraph as ig

In [44]:
edges_data = df_for_leiden_with_weight_filtered.select([
    "source_node", "target_node", "weight"
]).collect().to_numpy()

In [50]:
g = ig.Graph.TupleList(edges_data, directed=True, weights=True)

In [52]:
partition = g.community_leiden(weights='weight', objective_function='modularity')

In [53]:
partition

In [54]:
# Create a mapping of Transaction ID -> Community ID
node_names = g.vs["name"]
community_ids = partition.membership

community_df = pl.DataFrame({
    "node_id_str": node_names,
    "community_id": community_ids
})

# Now you can join this back to your original transaction data to see the "Fraud Rings"

In [58]:
community_df

node_id_str,community_id
str,i64
"""8000955D08076887702022-09-08 1…",0
"""807688770807E0A9A02022-09-08 1…",0
"""807AD7960807AD6AA02022-09-09 1…",1
"""807AD6AA08086A3C502022-09-09 1…",1
"""8076DEBB0807C299502022-09-03 1…",2
…,…
"""8077CA2A08063DECB02022-09-05 2…",42017
"""801EE4A00810A99EB02022-09-09 1…",19061
"""805B92690806D0D5802022-09-02 0…",42018
